# Converting from Tensorflow to ONNX
    we will convert ssd resnet50 to onnx and apply the onnx model on image. 

In [1]:
import os
import sys
# change working directory
ROOT = os.path.dirname(os.getcwd())

# load GPU into environnement
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ['ROOT'] = ROOT

model = "pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model"
## Set needed paths
os.environ['MODEL'] = model
## onnx path
onnx_name = model.split('/')[1]
os.environ['ONNX'] = os.path.join(model.split('/')[0],model.split('/')[1],model.split('/')[1])
##  set onnx opset. ssd and mask suport 11,12,..
os.environ['OPSET'] = '11'
!cd $ROOT

## Check the input and ouptup of the model

In [2]:
!saved_model_cli show --dir $ROOT/$MODEL  --tag_set serve --signature_def serving_default

2021-04-01 10:06:51.864130: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
The given SavedModel SignatureDef contains the following input(s):
  inputs['input_tensor'] tensor_info:
      dtype: DT_UINT8
      shape: (1, -1, -1, 3)
      name: serving_default_input_tensor:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['detection_anchor_indices'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: StatefulPartitionedCall:0
  outputs['detection_boxes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 4)
      name: StatefulPartitionedCall:1
  outputs['detection_classes'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100)
      name: StatefulPartitionedCall:2
  outputs['detection_multiclass_scores'] tensor_info:
      dtype: DT_FLOAT
      shape: (1, 100, 91)
      name: StatefulPartitionedCall:3
  outputs['detection_scores'] tensor_info:
      dtype: DT_FLO

## Convert to onnx using saved_model and a frozen model
    We don't a frozen Model because tensorflow 2.x no longer not provide an convertor to freeze the model
    the supported opset for ssd and mask are 11,12,..

In [3]:
!python -m tf2onnx.convert --opset $OPSET --fold_const --saved-model $ROOT/$MODEL --output $ROOT/$ONNX.onnx

2021-04-01 10:07:20.424785: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-01 10:07:22,238 - WARNING - '--tag' not specified for saved_model. Using --tag serve
2021-04-01 10:07:23.376878: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-01 10:07:23.433095: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-04-01 10:07:23.433183: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-01 10:07:23.503451: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-04-01 10:07:23.503611: I tensorflow/st

2021-04-01 10:08:11.384940: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:933] Optimization results for grappler item: graph_to_optimize
  constant_folding: Graph size after: 4994 nodes (-3655), 6366 edges (-4177), time = 2847.625ms.
  function_optimizer: function_optimizer did nothing. time = 10.606ms.
  constant_folding: Graph size after: 4994 nodes (0), 6366 edges (0), time = 383.158ms.
  function_optimizer: function_optimizer did nothing. time = 7.143ms.

2021-04-01 10:08:15.271093: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-04-01 10:08:15.273881: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1902] Adding visible gpu devices: 0
2021-04-01 10:08:15.273972: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1300] Device interconnect StreamExecutor with

2021-04-01 10:08:23.211569: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1446] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 4611 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1)
2021-04-01 10:08:23.253127: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-04-01 10:08:23.254310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1902] Adding visible gpu devices: 0
2021-04-01 10:08:23.254379: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1300] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-04-01 10:08:23.254403: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1306]      0 
2021-04-01 10:08:23.254425: I tensorflow/core/common_runtime/gpu/gpu_de

2021-04-01 10:08:23.398143: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1760] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 6GB computeCapability: 6.1
coreClock: 1.7085GHz coreCount: 10 deviceMemorySize: 5.93GiB deviceMemoryBandwidth: 178.99GiB/s
2021-04-01 10:08:23.398730: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1902] Adding visible gpu devices: 0
2021-04-01 10:08:23.398770: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1300] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-04-01 10:08:23.398783: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1306]      0 
2021-04-01 10:08:23.398795: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1319] 0:   N 
2021-04-01 10:08:23.399458: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1446] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 4611 MB memory) -> physical GPU (device: 0, name: GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, com

2021-04-01 10:08:40,139 - INFO - folding node using tf type=ExpandDims, name=StatefulPartitionedCall/Postprocessor/ExpandDims_2
2021-04-01 10:08:40,139 - INFO - folding node using tf type=ConcatV2, name=StatefulPartitionedCall/MultiscaleGridAnchorGenerator/GridAnchorGenerator/concat
2021-04-01 10:08:40,141 - INFO - folding node using tf type=Select, name=StatefulPartitionedCall/Postprocessor/BatchMultiClassNonMaxSuppression/PadOrClipBoxList/Select_1
2021-04-01 10:08:40,141 - INFO - folding node using tf type=Select, name=StatefulPartitionedCall/Postprocessor/BatchMultiClassNonMaxSuppression/PadOrClipBoxList/Select_8
2021-04-01 10:08:40,184 - INFO - folding node type=Range, name=StatefulPartitionedCall/Postprocessor/range
2021-04-01 10:08:43,967 - WARNING - ONNX Failed to infer shapes and dtypes for [StatefulPartitionedCall/Postprocessor/BatchMultiClassNonMaxSuppression/unstack_4__33, type: Squeeze]
Traceback (most recent call last):
  File "/home/kamgo/.virtualenvs/tfTrainOdIs/lib/pyth

2021-04-01 10:08:44,164 - WARNING - Node StatefulPartitionedCall/ResNet50V1_FPN/functional_1/conv4_block2_1_bn/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:44,182 - WARNING - Node StatefulPartitionedCall/ResNet50V1_FPN/functional_1/conv4_block2_2_bn/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:44,188 - WARNING - Node StatefulPartitionedCall/ResNet50V1_FPN/functional_1/conv4_block2_3_bn/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:44,193 - WARNING - Node StatefulPartitionedCall/ResNet50V1_FPN/functional_1/conv4_block3_1_bn/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save th

2021-04-01 10:08:44,936 - WARNING - Node StatefulPartitionedCall/WeightSharedConvolutionalBoxPredictor/BoxPredictionTower/conv2d_2/BatchNorm/feature_2/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:44,956 - WARNING - Node StatefulPartitionedCall/WeightSharedConvolutionalBoxPredictor/BoxPredictionTower/conv2d_3/BatchNorm/feature_2/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:44,977 - WARNING - Node StatefulPartitionedCall/ResNet50V1_FPN/bottom_up_block5_batchnorm/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:44,996 - WARNING - Node StatefulPartitionedCall/WeightSharedConvolutionalBoxPredictor/ClassPredictionTower/conv2d_0/BatchNorm/feat

2021-04-01 10:08:45,568 - WARNING - Node StatefulPartitionedCall/ResNet50V1_FPN/FeatureMaps/top_down/smoothing_1_batchnorm/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:45,586 - WARNING - Node StatefulPartitionedCall/WeightSharedConvolutionalBoxPredictor/ClassPredictionTower/conv2d_0/BatchNorm/feature_0/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:45,605 - WARNING - Node StatefulPartitionedCall/WeightSharedConvolutionalBoxPredictor/ClassPredictionTower/conv2d_1/BatchNorm/feature_0/FusedBatchNormV3 of type FusedBatchNormV3 has is_training set to true, which is not supperted. Please re-save the model with training set to false.
2021-04-01 10:08:45,624 - WARNING - Node StatefulPartitionedCall/WeightSharedConvolutionalBoxPredictor/ClassPredictionTower/con

2021-04-01 10:08:47,409 - INFO - Optimizing ONNX model
2021-04-01 10:09:38,378 - INFO - After optimization: BatchNormalization -94 (97->3), Cast -656 (1254->598), Const -3135 (3574->439), Identity -66 (66->0), Mul -2 (220->218), ReduceSum -90 (91->1), Reshape -90 (292->202), Shape -91 (195->104), Slice -1 (293->292), Squeeze -2 (214->212), Sub -90 (201->111), Transpose -401 (427->26), Unsqueeze -101 (228->127)
2021-04-01 10:09:38,761 - INFO - 
2021-04-01 10:09:38,762 - INFO - Successfully converted TensorFlow model /home/kamgo/Dokumente/Projects/Train_tensorflow_OD_API/workspace/TF_Train_OD_IS/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model to ONNX
2021-04-01 10:09:40,884 - INFO - ONNX model is saved at /home/kamgo/Dokumente/Projects/Train_tensorflow_OD_API/workspace/TF_Train_OD_IS/pre_trained_models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.onnx
